In [64]:
# Importamos las librerías:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

In [65]:
# Consumimos la base de datos y extraemos sólo las columnas necesarias:
def primerafuncion():
    link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
    data = pd.read_excel(link,'TODO CORREGIMIENTO')

    # eliminamos filas que no importan y asignamos la primera como cabecera
    data.drop([0], inplace=True)
    data.columns = data.iloc[0]
    data.drop([1], inplace=True)
    # data.head(10)

    # extraemos solo las columnas que nos interesan
    our_data = data[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]

    return(our_data)  

In [66]:
data = primerafuncion()
data

1,CD_Corr,Corregimiento,Fecha,Nuevos Confirmados,Nuevos Fallecidos,Nuevos Recuperados,Población
2,80817,Pacora,2020-06-05 00:00:00,NaN,NaN,NaN,35056
3,80819,Tocumen,2020-06-05 00:00:00,NaN,NaN,NaN,91447
4,130101,Arraiján (Cabecera),2020-06-05 00:00:00,NaN,NaN,NaN,53634
5,80821,24 de Diciembre,2020-06-05 00:00:00,NaN,NaN,NaN,79798
6,81002,Belisario Porras,2020-06-05 00:00:00,NaN,NaN,NaN,55951
...,...,...,...,...,...,...,...
70767,70314,Villa Lourdes,2020-09-17 00:00:00,0,0,0,1088
70768,20107,Virgen del Carmen,2020-09-17 00:00:00,0,0,0,10367
70769,50206,Yape,2020-09-17 00:00:00,0,0,0,216
70770,50317,Zapallal,2020-09-17 00:00:00,0,0,0,2072


In [67]:
# Extrae solo un corregimiento definido en el orden i del vector que rescata todos los corregimientos
# en forma única: np.unique(data['CD_Corr']):

CD_Corr_unique = np.unique(data['CD_Corr'])

def segundafuncion(i):
    
    # ***********************************************************************
    # 1 aca debiese trabajar el ciclo for al integrar segundafuncion(i) a la funcionglobal().
    # 2 data = primerafuncion() debe estar en la funcionglobal() fuera del ciclo for.
    # 3 la línea: CD_Corr_unique = np.unique(data['CD_Corr']) debe salir de acá.
    # e incluirse fuera del ciclo for en la funcionglobal().
    # ***********************************************************************

    
    our_data_filter = data[data['CD_Corr'] == CD_Corr_unique[i]]

    our_data_filter.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)
    our_data_filter['NuevosConfirmados'] = pd.to_numeric(our_data_filter['NuevosConfirmados'])
    our_data_filter['NuevosFallecidos'] = pd.to_numeric(our_data_filter['NuevosFallecidos'])
    our_data_filter['NuevosRecuperados'] = pd.to_numeric(our_data_filter['NuevosRecuperados'])
    our_data_filter['Poblacion'] = pd.to_numeric(our_data_filter['Poblacion'])

    #eliminamos la primera fila que es un NA
    our_data_filter = our_data_filter.iloc[1:]

    our_data_filter['NuevosConfirmados'] = our_data_filter['NuevosConfirmados']+our_data_filter['NuevosFallecidos']
    our_data_filter['NuevosConfirmadosacum'] = our_data_filter['NuevosConfirmados'].cumsum()
    our_data_filter['NuevosRecuperadosacum'] = our_data_filter['NuevosRecuperados'].cumsum()
    our_data_filter["Susceptibles"] = our_data_filter['Poblacion']-our_data_filter['NuevosConfirmadosacum']-our_data_filter['NuevosRecuperadosacum']
    our_data_filter["pob_log_Susceptibles"] = our_data_filter['Poblacion']*np.log(our_data_filter["Susceptibles"])

    # extraemos un subconjunto que llegue solo hasta el dia de hoy:
    now = datetime.datetime.now()
    our_data_filter_un = our_data_filter[our_data_filter['Fecha'] < now]

    # filtramos desde el primer dia en el que haya recuperados:
    our_data_filter_un_final = our_data_filter_un[our_data_filter_un['NuevosRecuperadosacum'] > 0]
    
    y = our_data_filter_un_final["pob_log_Susceptibles"]

    return(our_data_filter_un_final)

In [68]:
# por ejemplo el corregimiento en el lugar 100 del vector
data2 = segundafuncion(100)
data2

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles
9155,30303,El Guásimo,2020-06-18 00:00:00,3.0,0.0,1.0,3230,3.0,1.0,3226.0,26095.164376
9829,30303,El Guásimo,2020-06-19 00:00:00,1.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980
10503,30303,El Guásimo,2020-06-20 00:00:00,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980
11177,30303,El Guásimo,2020-06-21 00:00:00,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980
11851,30303,El Guásimo,2020-06-22 00:00:00,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980
...,...,...,...,...,...,...,...,...,...,...,...
46899,30303,El Guásimo,2020-08-13 00:00:00,11.0,0.0,0.0,3230,80.0,24.0,3126.0,25993.455720
47573,30303,El Guásimo,2020-08-14 00:00:00,1.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286
48247,30303,El Guásimo,2020-08-15 00:00:00,0.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286
48921,30303,El Guásimo,2020-08-16 00:00:00,0.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286


In [69]:
# Calcula el R0 para el dataset en cuestion:

def tercerafuncion(data2):
    
    # declaramos una lista para almacenar los R0:
    lista_R0 = []
    
    # Reemplazamos los NA por 0:
    data2.fillna("0", inplace = True) 
    
    y = data2["pob_log_Susceptibles"]
    for i in range(len(y)):
        try:
            y = data2["pob_log_Susceptibles"]
            x = data2["NuevosRecuperadosacum"]
            yy = y.head(i+1)
            xx = x.head(i+1)
            # print(xx)
            # print(yy)
            # Creamos el objeto de Regresión Lineal
            regresion_lineal = linear_model.LinearRegression()

            regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
            r0 = -regresion_lineal.coef_
            lista_R0.append(r0)
            
        except:
            print("Error")
            lista_R0.append(0)
            
    data2["r0"] = lista_R0
    
    return(data2)

In [70]:
data3 = tercerafuncion(data2)
data3

1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0
9155,30303,El Guásimo,2020-06-18,3.0,0.0,1.0,3230,3.0,1.0,3226.0,26095.164376,[[-0.0]]
9829,30303,El Guásimo,2020-06-19,1.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980,[[-0.0]]
10503,30303,El Guásimo,2020-06-20,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980,[[-0.0]]
11177,30303,El Guásimo,2020-06-21,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980,[[-0.0]]
11851,30303,El Guásimo,2020-06-22,0.0,0.0,0.0,3230,4.0,1.0,3225.0,26094.162980,[[-0.0]]
...,...,...,...,...,...,...,...,...,...,...,...,...
46899,30303,El Guásimo,2020-08-13,11.0,0.0,0.0,3230,80.0,24.0,3126.0,25993.455720,[[3.652039656352653]]
47573,30303,El Guásimo,2020-08-14,1.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286,[[3.699715510738408]]
48247,30303,El Guásimo,2020-08-15,0.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286,[[3.7401636226005324]]
48921,30303,El Guásimo,2020-08-16,0.0,0.0,0.0,3230,81.0,24.0,3125.0,25992.422286,[[3.774911790966846]]


In [71]:
def cuartafuncion(data3): 
    data3.to_csv (r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama.csv', header=False, mode='a', index = False)
    

In [78]:
def quintafuncion(): 
    data_second_r0 = pd.read_csv(r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama.csv') 
    data_second_r0.columns = ['Código del Corregimiento','Corregimiento','Fecha','Casos Confirmados','Nuevos Fallecidos','Nuevos Recuperados','Poblacion','Nuevos Confirmados acum','Nuevos Recuperados acum','Susceptibles','pob_log_Susceptibles','r0']
    x = data_second_r0['r0'].str.replace("]]", "")
    y = x.str.replace("[", "")
    z_dataframe = pd.DataFrame(y)
    del data_second_r0['r0']
    data_second_r0['r0'] = z_dataframe
    
    f = open(r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama_final.csv', "w")
    f.truncate()
    f.close()
    
    data_second_r0.to_csv(r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama_final.csv', header=True, mode='a', index = False)
    return data_second_r0

In [58]:
def anadecabecera(data):
    data.columns = ['Código del Corregimiento','Corregimiento','Fecha','Casos Confirmados','Nuevos Fallecidos','Nuevos Recuperados','Poblacion','Nuevos Confirmados acum','Nuevos Recuperados acum','Susceptibles','pob_log_Susceptibles','r0']
    data.to_csv(r'C:\Users\usuario\Desktop\GitHub-trabajo\Bases_de_Datos_DS\R0_Panama_final_final.csv', header=True, mode='a', index = False)


In [79]:
def funcionglobal():
    data = primerafuncion()
    cant_de_corr = np.unique(data['CD_Corr'])
    for i in range(len(cant_de_corr)):
    #for i in range(1):
        cuartafuncion(tercerafuncion(segundafuncion(i)))
    datas = quintafuncion()
    #anadecabecera(datas)

In [80]:
funcionglobal()

Error
Error
Error
Error
Error
Error
Error
Error
Error
